<a href="https://colab.research.google.com/github/unreal-kz/Notes/blob/master/FineTuneSAM_SEG_ECHO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets segments-ai evaluate
!pip install -U accelerate
!pip install -U transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install ipywidgets

In [2]:
# fc54be4c5ea286d6edd6ad27a963a73a91e5ab1b
from segments import SegmentsClient
from getpass import getpass

api_key = getpass('Enter your API key: ')
segments_client = SegmentsClient(api_key)

KeyboardInterrupt: 

In [14]:
from segments.huggingface import release2dataset

# dataset_identifier = 'label_h/Test_seg'
dataset_identifier = 'ai_segment/Real_ECHO_Dateset'
release_name = 'labeled_331'

release = segments_client.get_release(dataset_identifier, release_name)
hf_dataset = release2dataset(release)

Map:   0%|          | 0/331 [00:00<?, ? examples/s]

Map:   0%|          | 0/331 [00:00<?, ? examples/s]

In [15]:
hf_dataset

Dataset({
    features: ['name', 'uuid', 'status', 'image', 'label.annotations', 'label.segmentation_bitmap'],
    num_rows: 331
})

In [16]:
from segments.utils import get_semantic_bitmap


def convert_segmentation_bitmap(example):
    return {
        "label.segmentation_bitmap":
            get_semantic_bitmap(
                example["label.segmentation_bitmap"],
                example["label.annotations"],
            )
    }


semantic_dataset = hf_dataset.map(
    convert_segmentation_bitmap,
)

Map:   0%|          | 0/331 [00:00<?, ? examples/s]

In [17]:
semantic_dataset = semantic_dataset.rename_column('image', 'pixel_values')
semantic_dataset = semantic_dataset.rename_column('label.segmentation_bitmap', 'label')
semantic_dataset = semantic_dataset.rename_column('label.annotations', 'annotation')
semantic_dataset = semantic_dataset.remove_columns(['uuid', 'status'])

In [1]:
hf_dataset_identifier = "unreal-hug/REAL_DATASET_SEG_331_w_annotaion_and_name_10_lb"
# hf_dataset_identifier = "unreal-hug/REAL_DATASET_SEG_331_w_annotaion_and_name"
# semantic_dataset.push_to_hub(hf_dataset_identifier)

In [2]:
from datasets import load_dataset

ds = load_dataset(hf_dataset_identifier, split='all')

In [3]:
ds

Dataset({
    features: ['name', 'pixel_values', 'annotation', 'label'],
    num_rows: 331
})

In [4]:
myData = ds.train_test_split()

In [6]:
myData["train"]

Dataset({
    features: ['name', 'pixel_values', 'annotation', 'label'],
    num_rows: 248
})

In [10]:
cnt = 0
myCategoryDict = {}
while cnt < ds.num_rows:
    for i in ds[cnt]['annotation']:
        if i['category_id'] in myCategoryDict.keys():
            myCategoryDict[i['category_id']] += 1
        else:
            myCategoryDict[i['category_id']] = 1
    cnt+=1

myCategoryDict

{1: 319, 4: 299, 9: 233, 2: 279, 3: 271, 7: 99, 11: 322, 10: 243, 5: 74, 6: 5}

In [11]:
import random
# random.seed()
rand_ss = set()
for _ in range(20):
    rand_ss.add(random.randint(1,1000))
rand_ss,len(rand_ss)

({22,
  35,
  83,
  203,
  218,
  253,
  286,
  301,
  504,
  513,
  533,
  595,
  619,
  638,
  682,
  783,
  853,
  876,
  880,
  940},
 20)

### Id2label: {0: 'unlabeled', 1: 'LV', 2: 'RV', 3: 'RA', 4: 'LA', 5: 'VS', 6: 'AS', 7: 'MK', 8: 'TK', 9: 'ASD', 10: 'VSD', 11: 'AK'}

### Id2label: {0: 'unlabeled', 1: 'LV', 2: 'RV', 3: 'RA', 4: 'LA', 5: 'ASD', 6: 'VSD', 7: 'TK', 9: 'MK', 10: 'AS'}

In [12]:
seed_bucket = {}

for ss in rand_ss:
    dict_ids = {}
    is_zero = False
    for i in range(11):
        if i != 8:
            dict_ids[i] = 0
        
    ds = load_dataset(hf_dataset_identifier, split='all')
    ds = ds.shuffle(seed=ss)
    ds = ds.train_test_split(test_size=0.2)
    train_ds = ds["train"]
    for i in range(train_ds.num_rows):
        for mydict in train_ds[i]["annotation"]:
            if mydict['category_id'] in dict_ids:
                dict_ids[mydict['category_id']] += 1
    print(dict_ids)
    for key, val  in dict_ids.items():
        if key != 0 and val == 0:
            is_zero = True
            break
    if is_zero == False:
        seed_bucket[ss] = dict_ids
    


{0: 0, 1: 253, 2: 223, 3: 217, 4: 240, 5: 60, 6: 5, 7: 79, 9: 184, 10: 189}
{0: 0, 1: 254, 2: 224, 3: 216, 4: 240, 5: 62, 6: 3, 7: 78, 9: 192, 10: 199}
{0: 0, 1: 254, 2: 223, 3: 215, 4: 239, 5: 61, 6: 5, 7: 83, 9: 181, 10: 195}
{0: 0, 1: 254, 2: 221, 3: 215, 4: 239, 5: 57, 6: 4, 7: 79, 9: 190, 10: 192}
{0: 0, 1: 253, 2: 224, 3: 217, 4: 237, 5: 58, 6: 2, 7: 74, 9: 187, 10: 193}
{0: 0, 1: 256, 2: 221, 3: 218, 4: 240, 5: 57, 6: 5, 7: 82, 9: 191, 10: 196}
{0: 0, 1: 256, 2: 222, 3: 213, 4: 242, 5: 58, 6: 4, 7: 80, 9: 179, 10: 197}
{0: 0, 1: 253, 2: 218, 3: 215, 4: 238, 5: 60, 6: 4, 7: 82, 9: 180, 10: 196}
{0: 0, 1: 257, 2: 223, 3: 213, 4: 243, 5: 61, 6: 5, 7: 82, 9: 182, 10: 195}
{0: 0, 1: 258, 2: 222, 3: 216, 4: 241, 5: 58, 6: 5, 7: 81, 9: 188, 10: 188}
{0: 0, 1: 255, 2: 223, 3: 217, 4: 240, 5: 61, 6: 2, 7: 80, 9: 182, 10: 194}
{0: 0, 1: 254, 2: 223, 3: 216, 4: 239, 5: 60, 6: 3, 7: 76, 9: 184, 10: 201}
{0: 0, 1: 256, 2: 219, 3: 211, 4: 237, 5: 59, 6: 4, 7: 80, 9: 177, 10: 190}
{0: 0, 1: 25

In [17]:
seed_bucket.items()

dict_items([(513, {0: 0, 1: 253, 2: 223, 3: 217, 4: 240, 5: 60, 6: 5, 7: 79, 9: 184, 10: 189}), (783, {0: 0, 1: 254, 2: 224, 3: 216, 4: 240, 5: 62, 6: 3, 7: 78, 9: 192, 10: 199}), (533, {0: 0, 1: 254, 2: 223, 3: 215, 4: 239, 5: 61, 6: 5, 7: 83, 9: 181, 10: 195}), (22, {0: 0, 1: 254, 2: 221, 3: 215, 4: 239, 5: 57, 6: 4, 7: 79, 9: 190, 10: 192}), (286, {0: 0, 1: 253, 2: 224, 3: 217, 4: 237, 5: 58, 6: 2, 7: 74, 9: 187, 10: 193}), (35, {0: 0, 1: 256, 2: 221, 3: 218, 4: 240, 5: 57, 6: 5, 7: 82, 9: 191, 10: 196}), (682, {0: 0, 1: 256, 2: 222, 3: 213, 4: 242, 5: 58, 6: 4, 7: 80, 9: 179, 10: 197}), (940, {0: 0, 1: 253, 2: 218, 3: 215, 4: 238, 5: 60, 6: 4, 7: 82, 9: 180, 10: 196}), (301, {0: 0, 1: 257, 2: 223, 3: 213, 4: 243, 5: 61, 6: 5, 7: 82, 9: 182, 10: 195}), (203, {0: 0, 1: 258, 2: 222, 3: 216, 4: 241, 5: 58, 6: 5, 7: 81, 9: 188, 10: 188}), (595, {0: 0, 1: 255, 2: 223, 3: 217, 4: 240, 5: 61, 6: 2, 7: 80, 9: 182, 10: 194}), (83, {0: 0, 1: 254, 2: 223, 3: 216, 4: 239, 5: 60, 6: 3, 7: 76, 9:

In [26]:
ds

DatasetDict({
    train: Dataset({
        features: ['name', 'pixel_values', 'annotation', 'label'],
        num_rows: 264
    })
    test: Dataset({
        features: ['name', 'pixel_values', 'annotation', 'label'],
        num_rows: 67
    })
})

In [ ]:
train_ds = ds["train"]
test_ds = ds["test"]

In [32]:
for i in train_ds[1]['annotation']:
    print(i['category_id'])

{'id': 1, 'category_id': 1}
{'id': 2, 'category_id': 9}
{'id': 3, 'category_id': 4}
{'id': 4, 'category_id': 11}
{'id': 5, 'category_id': 10}
{'id': 6, 'category_id': 2}
{'id': 7, 'category_id': 3}


In [ ]:
from torchvision.transforms import ColorJitter, RandomRotation, RandomHorizontalFlip
import torchvision.transforms.functional as TF
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import torch

class NewProcessor:
    def __init__(self):
        self.jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 
        self.flip = RandomHorizontalFlip(p=0.5)
        self.rotation = RandomRotation(degrees=[-15,15], center=[128, 20])
        
    def __call__(self, image, label):
        image = TF.pil_to_tensor(self.jitter(image)) # size [3, 256, 256], unit8
        label = TF.pil_to_tensor(label) # [1, 256, 256], uint32
        # print(image.dtype, label.dtype)
        merged = torch.cat((image, label), dim=0) # [4, 256, 256]
        merged = self.flip(merged) 
        merged = self.rotation(merged)
        image = TF.to_pil_image(merged[:3].to(dtype=torch.uint8))
        label = TF.to_pil_image(merged[3])
        return image, label

In [ ]:
image = ds['train'][2]['pixel_values']
mask = ds['train'][2]['label']
new_transforms = NewProcessor()
new_image, new_mask = new_transforms(image, mask)

In [ ]:
image

In [ ]:
plt.imshow(np.array(mask)/13)

In [ ]:
import numpy as np
from PIL import Image

def adjust_contrast(image_path):
    # Load the image in I mode
    img = image_path

    # Convert to numpy array for manipulation
    img_array = np.array(img)

    # Normalize the array to span the full 0-255 range of uint8
    if img_array.max() != img_array.min():  # Avoid division by zero if image is uniform
        img_normalized = 255 * (img_array - img_array.min()) / (img_array.max() - img_array.min())
    else:
        img_normalized = np.zeros(img_array.shape)

    # Convert the normalized array back to a PIL image
    img_normalized = Image.fromarray(img_normalized.astype(np.uint8))

    # Convert to 'L' mode to ensure it's treated as grayscale
    img_normalized = img_normalized.convert('L')

    # Display the image
    img_normalized.show()
    return img_normalized
    # Optionally, save the image
    # img_normalized.save('adjusted_image.png')

# Specify the path to your 'I' mode image
image_path = mask
image_gray = adjust_contrast(image_path)


In [ ]:
mask.mode

In [ ]:
np.array(mask)/13

In [ ]:
image.mode

In [ ]:
new_image

In [ ]:
# image_gray = Image.fromarray(np.array(mask))
image_gray.mode

In [ ]:
from PIL import Image

def overlay_mask(image_path, mask_path, mask_opacity=255):
    # Load the original image
    image = image_path.convert("RGBA")
    
    # Load the mask
    mask = mask_path
    
    # Convert the mask to RGBA if necessary
    if mask.mode != 'RGBA':
        # mask = mask.convert('L')  # Convert to grayscale
        mask = Image.merge('RGBA', (mask, mask, mask, mask))
    
    # Adjust mask transparency
    # Set the alpha channel to the desired opacity
    r, g, b, alpha = mask.split()
    r = r.point(lambda i: i * mask_opacity / 128)
    g = g.point(lambda i: i * mask_opacity / 128)
    b = b.point(lambda i: i * mask_opacity / 128)
    alpha = alpha.point(lambda i: i * mask_opacity / 50)
    mask = Image.merge('RGBA', (r, g, b, alpha))
    
    # Composite the mask with the image
    combined = Image.alpha_composite(image, mask)
    
    # Convert back to RGB and save the output
    combined = combined.convert("RGB")
    # combined.save(output_path)
    combined.show()

# Specify the paths to your image and mask
image_path = image
mask_path = image_gray
# output_path = 'path_to_the_output_image.jpg'

# Overlay the mask over the image
overlay_mask(image_path, mask_path)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def overlay_mask(image_path, mask_path):
    # Load the image and mask
    image = image_path.convert('RGBA')
    mask = mask_path

    # Generate a color map
    cmap = plt.get_cmap('tab20')  # This cmap provides 20 distinct colors

    # Convert the mask to a color image
    mask_array = np.array(mask)
    colored_mask = np.zeros((*mask_array.shape, 4), dtype=np.uint8)

    # Map each label to a color
    unique_labels = np.unique(mask_array)
    for label in unique_labels:
        if label != 0:  # Assuming 0 is the background and should be transparent
            color = cmap(label / len(unique_labels))  # Normalize label ID to the cmap size
            colored_mask[mask_array == label] = [int(x * 255) for x in color[:3]] + [100]  # Set alpha

    # Convert arrays to PIL Image
    colored_mask_image = Image.fromarray(colored_mask, 'RGBA')

    # Overlay the mask on the image
    combined = Image.alpha_composite(image, colored_mask_image)

    # Convert back to RGB and show/save
    combined = combined.convert("RGB")
    combined.show()
    # combined.save('overlayed_image.png')

# Paths to your image and mask
image_path = image
mask_path = image_gray

overlay_mask(image_path, mask_path)


In [ ]:
from PIL import Image
import numpy as np

def inspect_image(file_path):
    # Load the image
    img = Image.open(file_path)

    # Display basic information about the image
    print(f"Format: {img.format}")  # Output format (e.g., JPEG, PNG)
    print(f"Mode: {img.mode}")  # Output mode (e.g., L, RGB)
    print(f"Size: {img.size}")  # Output size (width, height)

    # Convert the image to a numpy array for detailed analysis
    img_array = np.array(img)

    # Print array properties
    print(f"Array Shape: {img_array.shape}")  # Shape of the array
    print(f"Data Type: {img_array.dtype}")  # Data type of the array elements

    # Calculate and print unique values and their counts
    unique, counts = np.unique(img_array, return_counts=True)
    unique_counts = dict(zip(unique, counts))
    print("Unique pixel values and counts:")
    for value, count in unique_counts.items():
        print(f"Value: {value}, Count: {count}")

# Specify the path to your image file
file_path = 'path_to_your_image_file.png'
inspect_image(file_path)


In [20]:
import json
from huggingface_hub import hf_hub_download

filename = "id2label.json"
id2label = json.load(open(hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"), "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)
print("Id2label:", id2label)

id2label.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Id2label: {0: 'unlabeled', 1: 'LV', 2: 'RV', 3: 'RA', 4: 'LA', 5: 'ASD', 6: 'VSD', 7: 'TK', 9: 'MK', 10: 'AS'}


c:\Users\rogbeast\anaconda3\envs\hugging_face\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rogbeast\.cache\huggingface\hub\datasets--unreal-hug--REAL_DATASET_SEG_331_w_annotaion_and_name_10_lb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
from torchvision.transforms import ColorJitter
from transformers import (
    SegformerImageProcessor,
)

processor = SegformerImageProcessor()
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

def train_transforms(example_batch):
    images = [jitter(x) for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = processor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = processor(images, labels)
    return inputs


# Set transforms
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [ ]:
from transformers import SegformerForSemanticSegmentation

pretrained_model_name = "nvidia/mit-b0"
model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id
)

In [ ]:
from transformers import TrainingArguments

epochs = 25
lr = 0.0001
batch_size = 2

hub_model_id = "segformer-b0-finetuned-segments-ECHO-jan-20-v1"

training_args = TrainingArguments(
    "segformer-b0-finetuned-segments-ECHO-outputs",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id=hub_model_id,

    hub_strategy="end",
)

In [ ]:
import torch
from torch import nn
import evaluate
import multiprocessing

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,
            reduce_labels=processor.do_reduce_labels,
        )

    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

    return metrics

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
kwargs = {
    "tags": ["vision", "image-segmentation"],
    "finetuned_from": pretrained_model_name,
    "dataset": hf_dataset_identifier,
}

processor.push_to_hub(hub_model_id)
trainer.push_to_hub(**kwargs)

In [ ]:
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation

processor = SegformerImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained(f"unreal-hug/{hub_model_id}")